# [Macrophage and neutrophil heterogeneity at single-cell spatial resolution in human inflammatory bowel disease](https://pubmed.ncbi.nlm.nih.gov/37495570/)
GEO: [GSE214695](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE214695)

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
    library(tibble)
})

In [2]:
source('scTools/R/process.geo.data.r')

In [3]:
seurat_objs <- process.GEO.data(GEOID = "GSE214695")

Found 1 file(s)

GSE214695_series_matrix.txt.gz



Removed file paths:


In [4]:
# Merge samples
obj <- merge(seurat_objs[[1]], 
c(seurat_objs[[2]], seurat_objs[[3]], seurat_objs[[4]], seurat_objs[[5]], seurat_objs[[6]], seurat_objs[[7]],
 seurat_objs[[8]], seurat_objs[[9]], seurat_objs[[10]], seurat_objs[[11]], seurat_objs[[12]], seurat_objs[[13]],
seurat_objs[[14]], seurat_objs[[15]], seurat_objs[[16]], seurat_objs[[17]], seurat_objs[[18]]))
save(obj, file = file.path("objects", "00.rda"))

Warning message:
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


In [ ]:
# Clean up data
obj <- JoinLayers(obj)
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
obj <- subset(obj, subset = nFeature_RNA > 350 & nFeature_RNA < 5000 & percent.mt < 10)
obj
save(obj, file = file.path("objects", "01.rda"))

In [ ]:
# obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
# save(obj, file=file.path("objects", "02.rda"))

In [ ]:
# obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
# obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
# obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
# obj <- FindClusters(obj, resolution = 0.5)
# save(obj, file=file.path("objects", "03.rda"))

In [21]:
load(file = file.path("objects", "01.rda"))

In [22]:
rownames(obj@meta.data) <- make.unique(gsub("[^A-Za-z]+", "", rownames(obj@meta.data)))

In [23]:
mdata <- read.csv("GSE214695/GSE214695_cell_annotation.csv.gz", sep = ",")
mdata$cell_id <- make.unique(mdata$cell_id)
rownames(mdata) <- mdata$cell_id
rownames(mdata) <- make.unique(gsub("[^A-Za-z]+", "", rownames(mdata)))

In [24]:
metadata <- merge(obj@meta.data, mdata, by=0, all.x=TRUE)
metadata<- select(metadata, c(Row.names, sample, annotation, nanostring_reference))

In [25]:
obj <- AddMetaData(obj, metadata = metadata, col.name = "sample")
obj <- AddMetaData(obj, metadata = metadata, col.name = "annotation")
obj <- AddMetaData(obj, metadata = metadata, col.name = "nanostring_reference")

In [26]:
rownames(obj@assays$RNA@cells) <- make.unique(gsub("[^A-Za-z]+", "", rownames(obj@assays$RNA@cells)))

In [27]:
Idents(obj) <- "RNA"
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
save(obj, file=file.path("objects", "02.rda"))

Running SCTransform on assay: RNA

Running SCTransform on layer: counts

vst.flavor='v2' set. Using model with fixed slope and excluding poisson genes.

Variance stabilizing transformation of count matrix of size 20560 by 26787

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 3000 cells

Found 85 outliers - those will be ignored in fitting/regularization step


Skip calculation of full residual matrix

Will not return corrected UMI because residual type is not set to 'pearson'

Calculating gene attributes

Wall clock passed: Time difference of 8.343577 secs

Setting min_variance based on median UMI:  0.04

Calculating variance for residuals of type pearson for 20560 genes

Determine variable features

Setting min_variance based on median UMI:  0.04

Calculating residuals of type pearson for 2000 genes



  |======================================================================| 100%


Computing corrected UMI count matrix

Centering data matrix

Getting residuals for block 1(of 9) for counts dataset

Getting residuals for block 2(of 9) for counts dataset

Getting residuals for block 3(of 9) for counts dataset

Getting residuals for block 4(of 9) for counts dataset

Getting residuals for block 5(of 9) for counts dataset

Getting residuals for block 6(of 9) for counts dataset

Getting residuals for block 7(of 9) for counts dataset

Getting residuals for block 8(of 9) for counts dataset

Getting residuals for block 9(of 9) for counts dataset

Centering data matrix

Finished calculating residuals for counts

Set default assay to SCT



In [28]:
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
save(obj, file=file.path("objects", "03.rda"))

PC_ 1 
Positive:  IGKC, IGHA1, JCHAIN, IGHA2, IGHG1, IGHG3, IGLC2, IGLC3, IGHG4, MZB1 
	   IGHG2, IGKV3-20, IGHGP, IGKV4-1, DERL3, IGLL5, IGKV1-5, IGHM, IGHV3-23, IGLV6-57 
	   IGKV3-11, IGLV3-1, HERPUD1, IGHV3-7, DNAAF1, XBP1, IGLV2-14, IGKV1-39, IGKV3-15, CD79A 
Negative:  DCN, LUM, COL3A1, IFITM3, COL1A2, ADAMDEC1, CALD1, C1S, CXCL14, A2M 
	   EGR1, CCL2, SOD2, COL1A1, TMEM176B, FOS, NNMT, C1R, MFAP4, S100A6 
	   RARRES2, IGFBP7, IER3, FN1, FTH1, FBLN1, CFH, COL6A1, SPARC, MEG3 
PC_ 2 
Positive:  HLA-DRA, LYZ, HLA-DPB1, HLA-DRB1, HLA-DPA1, TYROBP, FCER1G, AIF1, CCL5, FTH1 
	   HLA-DQA1, HLA-DQB1, HLA-DRB5, TMSB4X, CCL4, S100A9, IL1B, CD74, IL32, CXCL8 
	   BCL2A1, CXCR4, PLAUR, CD83, C15orf48, IL7R, CD52, CD3D, S100A8, PTPRC 
Negative:  IGKC, IGHA1, JCHAIN, DCN, LUM, COL3A1, COL1A2, ADAMDEC1, CALD1, C1S 
	   CXCL14, IGHA2, CCL2, COL1A1, A2M, C1R, MFAP4, NNMT, EGR1, FBLN1 
	   FN1, RARRES2, FOS, IFITM3, CFH, IGFBP7, MEG3, COL6A1, TCF21, TMEM176B 
PC_ 3 
Positive:  CCL5, IL32, CD3D, K

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 26787
Number of edges: 860336

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9473
Number of communities: 26
Elapsed time: 3 seconds


In [29]:
immune_cell_names <- c(
  'CD8 CTL', 'Tregs', 'M2', 'PC IgA IgM', 'PC IgA heat shock 1', 'PC IgA 1',
  'PC IgA heat shock 2', 'ThF', 'PC IgA 3', 'Memory B cell', 'PC IgA Lambda 1',
  'CD4 ANXA1', 'NK', 'PC IgA 2', 'Cycling T cells', 'PC IgA 4', 'CD4 naïve',
  'CD8 CTL TRM', 'S1PR1 T cells', 'Plasmablast IgA Lambda 2', 'ILC3', 'GC B cell',
  'B cell', 'T cells CCL20', 'IDA macrophage', 'M1 ACOD1', 'PC IgG 1',
  'CD8 FGFBP2', 'DCs CD1c', 'M0_Ribhi', 'M1 CXCL5', 'Ribhi T cells 1',
  'Activated endothelium', 'M0', 'Ribhi T cells 2', 'DCs CCL22', 'B cell Ribhi',
  'M1 CXCL5', 'Tuft cells', 'Eosinophils', 'Cycling cells 3', 'Myofibroblasts',
  'Naïve B cell', 'DCs CCL22_Ribhi'
)

In [30]:
obj <- SetIdent(obj, value = "annotation")
obj <- subset(obj, idents = immune_cell_names, invert = TRUE)

In [31]:
blueprint.ref <- celldex::BlueprintEncodeData()
monaco.ref <- celldex::MonacoImmuneData()

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



In [32]:
sce <- LayerData(obj)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj@meta.data$blueprint.main <- blueprint.main$pruned.labels
save(obj, file=file.path("objects", "04.rda"))

In [33]:
table(obj@meta.data$blueprint.main)


       Adipocytes           B-cells      CD4+ T-cells      CD8+ T-cells 
               23              5023              1125              1405 
               DC Endothelial cells  Epithelial cells      Erythrocytes 
               31                54               486                21 
      Fibroblasts               HSC     Keratinocytes       Macrophages 
              487               172                 8               127 
  Mesangial cells         Monocytes          Myocytes       Neutrophils 
                1               384                 5                36 
         NK cells         Pericytes 
              119                16 

In [34]:
obj <- SetIdent(obj, value = "blueprint.main")
remove_low_count_cells <- function(seurat_obj, metadata_column, threshold = 20) {

  # obj <- SetIdent(obj, value = metadata_column)

  total_counts <- table(seurat_obj@meta.data[[metadata_column]])

  low_count_cells <- names(total_counts[total_counts < threshold])

  seurat_obj <- subset(seurat_obj, idents = low_count_cells, invert = TRUE)

  return(seurat_obj)
}
obj <- remove_low_count_cells(seurat_obj = obj, metadata_column = "blueprint.main", threshold = 20)

In [35]:
table(obj@meta.data$blueprint.main)


       Adipocytes           B-cells      CD4+ T-cells      CD8+ T-cells 
               23              5023              1125              1405 
               DC Endothelial cells  Epithelial cells      Erythrocytes 
               31                54               486                21 
      Fibroblasts               HSC       Macrophages         Monocytes 
              487               172               127               384 
      Neutrophils          NK cells 
               36               119 

In [36]:
sce <- LayerData(obj)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj@meta.data$monaco.fine <- monaco.fine$pruned.labels
save(obj, file=file.path('objects', '05.rda'))

In [37]:
table(obj@meta.data$monaco.fine)


   Central memory CD8 T cells           Classical monocytes 
                          106                           264 
  Effector memory CD8 T cells             Exhausted B cells 
                          246                           283 
    Follicular helper T cells        Intermediate monocytes 
                          234                           159 
        Low-density basophils       Low-density neutrophils 
                          127                            45 
                   MAIT cells       Myeloid dendritic cells 
                          295                           453 
                Naive B cells             Naive CD4 T cells 
                           62                            27 
            Naive CD8 T cells          Natural killer cells 
                            4                            92 
      Non classical monocytes   Non-switched memory B cells 
                           24                           151 
           Non-Vd2 gd T

In [38]:
obj <- SetIdent(obj, value = "monaco.fine")
remove_low_count_cells <- function(seurat_obj, metadata_column, threshold = 20) {

  total_counts <- table(seurat_obj@meta.data[[metadata_column]])

  low_count_cells <- names(total_counts[total_counts < threshold])

  seurat_obj <- subset(seurat_obj, idents = low_count_cells, invert = TRUE)

  return(seurat_obj)
}
obj <- remove_low_count_cells(seurat_obj = obj, metadata_column = "monaco.fine", threshold = 20)

In [39]:
table(obj@meta.data$monaco.fine)


   Central memory CD8 T cells           Classical monocytes 
                          106                           264 
  Effector memory CD8 T cells             Exhausted B cells 
                          246                           283 
    Follicular helper T cells        Intermediate monocytes 
                          234                           159 
        Low-density basophils       Low-density neutrophils 
                          127                            45 
                   MAIT cells       Myeloid dendritic cells 
                          295                           453 
                Naive B cells             Naive CD4 T cells 
                           62                            27 
         Natural killer cells       Non classical monocytes 
                           92                            24 
  Non-switched memory B cells            Non-Vd2 gd T cells 
                          151                           121 
                 Plasma

In [40]:
# Genes Rik was interested
genes <- 
c("LILRB1", "LILRB2", "LILRB3", "HLA-G", "IL1B",
 "IL6", "IL23A", "IL12A", "IL12B", "TNF", "IL10", "TIGIT", "PDCD1")

In [41]:
# Keep only genes present in data for `default.multigene`
seurat_data <- GetAssayData(object = obj)
seurat_genes <- rownames(seurat_data)
present_genes <- genes[genes %in% seurat_genes]

In [42]:
# Clean up metadata
obj@meta.data$Gender <- obj@meta.data$gender.ch1
obj@meta.data$Stim <- obj@meta.data$disease.state.ch1
obj@meta.data$Orig.Annotations <- obj@meta.data$annotation

In [43]:
columns_to_keep <-  c('orig.ident', 'nCount_RNA', 'Stim', 'Gender', 'Orig.Annotations', 'blueprint.main', 'monaco.fine')
obj@meta.data <- obj@meta.data[, columns_to_keep, drop = FALSE]
obj <- subset(obj, subset = Orig.Annotations != "NA")

Warning message:
“Removing 2119 cells missing data for vars requested”


In [44]:
save(obj, file=file.path('objects', '06.rda'))

In [45]:
# All stims
seu =  obj
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1,
             gene.mapping = TRUE,
             shiny.prefix = "sc1",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Crohn’s disease|healthy control|ulcerative colitis,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE
Gender,Gender,Female|Male,#A6CEE3|#B15928,1,TRUE,FALSE
Orig.Annotations,Orig.Annotations,BEST4 OTOP2|Colonocyte 1|Colonocyte 2|Cycling cells|Cycling myeloid|Cycling TA|DN EOMES|DN TNF|Endothelium|Enteroendocrine|Epithelium Ribhi|Fibroblasts Ribhi|FRCs|gd IEL|Glia|Goblet|IER fibroblasts|Inflammatory colonocyte|Inflammatory fibroblasts|Inflammatory monocytes|Laminin colonocytes|Lymphatic endothelium|M2.2|MAIT|Mast 1|Mast 2|Mast Ribhi|Mature goblet|MT fibroblasts|MT T cells|Neutrophil 1|Neutrophil 2|Neutrophil 3|Paneth-like|PC immediate early response|PC IgG 2|PC IGLL5|Pericytes|Plasmablast IgG|Plasmablast IgG Lambda|PLCG2 colonocytes|S1|S1 Ribhi|S1.2|S2a|S2b|S3|Secretory progenitor,#A6CEE3|#86B9D8|#66A5CD|#4791C2|#277DB7|#3889AC|#5AA1A3|#7CB999|#9FD18F|#A4D880|#86C969|#69BA53|#4BAC3E|#3B9F30|#6A9E4A|#999C63|#C79B7D|#F69A96|#F57E7E|#F06061|#EA4243|#E52426|#E63228|#EC593B|#F37F4F|#F9A662|#FDB965|#FDAA4B|#FE9B31|#FE8C17|#FD8004|#F18C36|#E59768|#D8A39A|#CCAFCC|#B79BCA|#A180BC|#8A64AE|#7449A0|#7D5599|#9F8399|#C2B099|#E5DD99|#FAF491|#E7CD77|#D5A65C|#C37F42|#B15928,12,TRUE,FALSE
blueprint.main,blueprint.main,Adipocytes|B-cells|CD4+ T-cells|CD8+ T-cells|DC|Endothelial cells|Epithelial cells|Erythrocytes|Fibroblasts|HSC|Macrophages|Monocytes|Neutrophils|NK cells,#A6CEE3|#3385BB|#84BF96|#6DBD57|#7F9D55|#F57C7C|#E42622|#FBB268|#FE8D19|#DE9E83|#9D7BBA|#977899|#F3E587|#B15928,4,TRUE,FALSE
monaco.fine,monaco.fine,Central memory CD8 T cells|Classical monocytes|Effector memory CD8 T cells|Exhausted B cells|Follicular helper T cells|Intermediate monocytes|Low-density basophils|Low-density neutrophils|MAIT cells|Myeloid dendritic cells|Naive B cells|Naive CD4 T cells|Natural killer cells|Non classical monocytes|Non-switched memory B cells|Non-Vd2 gd T cells|Plasmablasts|Plasmacytoid dendritic cells|Progenitor cells|Switched memory B cells|T regulatory cells|Terminal effector CD8 T cells|Th1 cells|Th1/Th17 cells|Th17 cells|Th2 cells|Vd2 gd T cells,#A6CEE3|#6CA9CF|#3385BB|#4693A8|#84BF96|#A3D77F|#6DBD57|#37A22F|#7F9D55|#D49B84|#F57C7C|#EB4647|#E42622|#F06C45|#FBB268|#FDA848|#FE8D19|#F48829|#DE9E83|#C6ADD3|#9D7BBA|#754AA0|#977899|#D6CA99|#F3E587|#D29F57|#B15928,7,TRUE,FALSE
PC_1,PC1,NA,NA,NA,FALSE,TRUE
PC_2,PC2,NA,NA,NA,FALSE,TRUE


In [46]:
# Ulcerative colitis
load(file=file.path('objects', '06.rda'))
obj <- subset(obj, subset = Stim == "ulcerative colitis")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)

save(obj, file=file.path('objects', 'uc.rda'))

seu =  obj
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2,
             gene.mapping = TRUE,
             shiny.prefix = "sc2",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

PC_ 1 
Positive:  IGKC, IGHA1, IGHG1, IGHG3, JCHAIN, IGHA2, IGHG4, IGLC2, IGHGP, IGLC3 
	   MZB1, IGKV3-20, IGHG2, DERL3, IGKV1-5, IGKV4-1, IGLV6-57, IGLV3-1, IGHV1-24, DNAAF1 
	   IGHV1-2, IGHV3-21, XBP1, IGHV3-23, IGLL5, IGHV3-30, BTG2, IGKV1-39, IGLV3-21, HERPUD1 
Negative:  HLA-DRA, SOD2, FCER1G, LYZ, HLA-DRB1, TIMP1, HLA-DPB1, S100A9, TYROBP, HLA-DPA1 
	   FTH1, AIF1, S100A8, S100A4, NAMPT, PLAUR, HLA-DRB5, IL1B, CXCL8, IFITM3 
	   BCL2A1, S100A6, C15orf48, HLA-DQB1, VCAN, CCL4, C5AR1, HLA-DQA1, GLUL, S100A11 
PC_ 2 
Positive:  IGKC, IGHG1, S100A9, LYZ, IGHG3, HLA-DRA, FCER1G, S100A8, TYROBP, AIF1 
	   IL1B, IGHA1, PLAUR, C5AR1, CXCL8, FCN1, SOD2, HLA-DRB1, IL1RN, CD14 
	   CD300E, C15orf48, EREG, IGHG4, NAMPT, HLA-DPA1, BCL2A1, HLA-DPB1, CCL3, TIMP1 
Negative:  CCL5, IL32, CD3D, GZMA, NKG7, KLRB1, TRBC1, GZMK, CD2, TRAC 
	   TRBC2, DCN, CD3G, LUM, CST7, TMSB4X, CALD1, CD3E, CXCL14, IGFBP7 
	   COL3A1, RARRES2, CD7, COL1A2, GZMB, C1S, CXCR4, IL7R, NNMT, CD52 
PC_ 3 
Positive:  IFI

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 3168
Number of edges: 89243

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9489
Number of communities: 23
Elapsed time: 0 seconds


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,ulcerative colitis,black,1,FALSE,FALSE
Gender,Gender,Female|Male,#A6CEE3|#B15928,1,TRUE,FALSE
Orig.Annotations,Orig.Annotations,BEST4 OTOP2|Colonocyte 1|Colonocyte 2|Cycling cells|Cycling myeloid|Cycling TA|DN EOMES|DN TNF|Endothelium|Enteroendocrine|Epithelium Ribhi|Fibroblasts Ribhi|FRCs|gd IEL|Glia|Goblet|IER fibroblasts|Inflammatory colonocyte|Inflammatory fibroblasts|Inflammatory monocytes|Laminin colonocytes|Lymphatic endothelium|M2.2|MAIT|Mast 1|Mast 2|Mast Ribhi|Mature goblet|MT fibroblasts|MT T cells|Neutrophil 1|Neutrophil 2|Neutrophil 3|Paneth-like|PC immediate early response|PC IgG 2|PC IGLL5|Pericytes|Plasmablast IgG|Plasmablast IgG Lambda|PLCG2 colonocytes|S1|S1 Ribhi|S1.2|S2a|S2b|S3|Secretory progenitor,#A6CEE3|#86B9D8|#66A5CD|#4791C2|#277DB7|#3889AC|#5AA1A3|#7CB999|#9FD18F|#A4D880|#86C969|#69BA53|#4BAC3E|#3B9F30|#6A9E4A|#999C63|#C79B7D|#F69A96|#F57E7E|#F06061|#EA4243|#E52426|#E63228|#EC593B|#F37F4F|#F9A662|#FDB965|#FDAA4B|#FE9B31|#FE8C17|#FD8004|#F18C36|#E59768|#D8A39A|#CCAFCC|#B79BCA|#A180BC|#8A64AE|#7449A0|#7D5599|#9F8399|#C2B099|#E5DD99|#FAF491|#E7CD77|#D5A65C|#C37F42|#B15928,12,TRUE,FALSE
blueprint.main,blueprint.main,Adipocytes|B-cells|CD4+ T-cells|CD8+ T-cells|DC|Endothelial cells|Epithelial cells|Erythrocytes|Fibroblasts|HSC|Macrophages|Monocytes|Neutrophils|NK cells,#A6CEE3|#3385BB|#84BF96|#6DBD57|#7F9D55|#F57C7C|#E42622|#FBB268|#FE8D19|#DE9E83|#9D7BBA|#977899|#F3E587|#B15928,4,TRUE,FALSE
monaco.fine,monaco.fine,Central memory CD8 T cells|Classical monocytes|Effector memory CD8 T cells|Exhausted B cells|Follicular helper T cells|Intermediate monocytes|Low-density basophils|Low-density neutrophils|MAIT cells|Myeloid dendritic cells|Naive B cells|Naive CD4 T cells|Natural killer cells|Non classical monocytes|Non-switched memory B cells|Non-Vd2 gd T cells|Plasmablasts|Plasmacytoid dendritic cells|Progenitor cells|Switched memory B cells|T regulatory cells|Terminal effector CD8 T cells|Th1 cells|Th1/Th17 cells|Th17 cells|Th2 cells|Vd2 gd T cells,#A6CEE3|#6CA9CF|#3385BB|#4693A8|#84BF96|#A3D77F|#6DBD57|#37A22F|#7F9D55|#D49B84|#F57C7C|#EB4647|#E42622|#F06C45|#FBB268|#FDA848|#FE8D19|#F48829|#DE9E83|#C6ADD3|#9D7BBA|#754AA0|#977899|#D6CA99|#F3E587|#D29F57|#B15928,7,TRUE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22,#A6CEE3|#62A3CB|#1F78B4|#68AB9F|#B2DF8A|#72BF5A|#33A02C|#969D62|#FB9A99|#EF595A|#E31A1C|#F06C45|#FDBF6F|#FE9E37|#FF7F00|#E4986B|#CAB2D6|#9A77B8|#6A3D9A|#B49D99|#FFFF99|#D8AC60|#B15928,6,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22,#A6CEE3|#62A3CB|#1F78B4|#68AB9F|#B2DF8A|#72BF5A|#33A02C|#969D62|#FB9A99|#EF595A|#E31A1C|#F06C45|#FDBF6F|#FE9E37|#FF7F00|#E4986B|#CAB2D6|#9A77B8|#6A3D9A|#B49D99|#FFFF99|#D8AC60|#B15928,6,TRUE,FALSE


In [47]:
# Crohn's disease
load(file=file.path('objects', '06.rda'))
obj <- subset(obj, subset = Stim == "Crohn’s disease")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)

save(obj, file=file.path('objects', 'cd.rda'))

seu =  obj
scConf3 = createConfig(seu)
makeShinyFiles(seu, scConf3,
             gene.mapping = TRUE,
             shiny.prefix = "sc3",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

PC_ 1 
Positive:  IGKC, IGHA1, JCHAIN, IGHA2, IGLC2, IGHG1, IGLC3, IGHG3, IGHG2, IGHM 
	   IGHG4, MZB1, IGKV3-20, IGLL5, IGKV4-1, IGHV3-23, DERL3, IGKV3-11, IGLV6-57, IGHV3-7 
	   IGKV1-5, IGLV2-14, IGHGP, DNAAF1, XBP1, HERPUD1, IGLV3-1, IGKV2-30, IGHV3-15, IGKV3-15 
Negative:  HLA-DRA, SOD2, LYZ, FTH1, CST3, IER3, TYROBP, FCER1G, AIF1, HLA-DPB1 
	   IL1B, CXCL8, HLA-DRB1, IFITM3, HLA-DPA1, PLAUR, S100A9, PTGS2, CCL3, COL3A1 
	   G0S2, CXCL2, CFD, CXCL14, NAMPT, EGR1, NFKBIA, C15orf48, DCN, BCL2A1 
PC_ 2 
Positive:  HLA-DRA, LYZ, TYROBP, AIF1, HLA-DPB1, FCER1G, HLA-DPA1, HLA-DRB1, IL1B, CXCL8 
	   FTH1, CCL4, S100A9, CCL3, HLA-DQA1, BCL2A1, HLA-DQB1, PLAUR, CD83, C1QA 
	   HLA-DRB5, CCL3L1, IL1RN, CD74, MS4A7, S100A8, CCL4L2, LST1, C1QB, GPR183 
Negative:  COL3A1, CXCL14, DCN, COL1A2, LUM, ADAMDEC1, C1S, CALD1, COL1A1, CCL2 
	   A2M, EGR1, CFD, RARRES2, NNMT, IGFBP7, IFITM3, C1R, COL6A1, MFAP4 
	   COL6A2, FOS, FBLN1, CFH, FN1, PTGDS, SPARCL1, TCIM, EDIL3, RND3 
PC_ 3 
Positive:  IL32,

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2495
Number of edges: 73895

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9277
Number of communities: 15
Elapsed time: 0 seconds


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Crohn’s disease,black,1,FALSE,FALSE
Gender,Gender,Female|Male,#A6CEE3|#B15928,1,TRUE,FALSE
Orig.Annotations,Orig.Annotations,BEST4 OTOP2|Colonocyte 1|Colonocyte 2|Cycling cells|Cycling myeloid|Cycling TA|DN EOMES|DN TNF|Endothelium|Enteroendocrine|Epithelium Ribhi|Fibroblasts Ribhi|FRCs|gd IEL|Glia|Goblet|IER fibroblasts|Inflammatory colonocyte|Inflammatory fibroblasts|Inflammatory monocytes|Laminin colonocytes|Lymphatic endothelium|M2.2|MAIT|Mast 1|Mast 2|Mast Ribhi|Mature goblet|MT fibroblasts|MT T cells|Neutrophil 1|Neutrophil 2|Neutrophil 3|Paneth-like|PC immediate early response|PC IgG 2|PC IGLL5|Pericytes|Plasmablast IgG|Plasmablast IgG Lambda|PLCG2 colonocytes|S1|S1 Ribhi|S1.2|S2a|S2b|S3|Secretory progenitor,#A6CEE3|#86B9D8|#66A5CD|#4791C2|#277DB7|#3889AC|#5AA1A3|#7CB999|#9FD18F|#A4D880|#86C969|#69BA53|#4BAC3E|#3B9F30|#6A9E4A|#999C63|#C79B7D|#F69A96|#F57E7E|#F06061|#EA4243|#E52426|#E63228|#EC593B|#F37F4F|#F9A662|#FDB965|#FDAA4B|#FE9B31|#FE8C17|#FD8004|#F18C36|#E59768|#D8A39A|#CCAFCC|#B79BCA|#A180BC|#8A64AE|#7449A0|#7D5599|#9F8399|#C2B099|#E5DD99|#FAF491|#E7CD77|#D5A65C|#C37F42|#B15928,12,TRUE,FALSE
blueprint.main,blueprint.main,Adipocytes|B-cells|CD4+ T-cells|CD8+ T-cells|DC|Endothelial cells|Epithelial cells|Erythrocytes|Fibroblasts|HSC|Macrophages|Monocytes|Neutrophils|NK cells,#A6CEE3|#3385BB|#84BF96|#6DBD57|#7F9D55|#F57C7C|#E42622|#FBB268|#FE8D19|#DE9E83|#9D7BBA|#977899|#F3E587|#B15928,4,TRUE,FALSE
monaco.fine,monaco.fine,Central memory CD8 T cells|Classical monocytes|Effector memory CD8 T cells|Exhausted B cells|Follicular helper T cells|Intermediate monocytes|Low-density basophils|Low-density neutrophils|MAIT cells|Myeloid dendritic cells|Naive B cells|Naive CD4 T cells|Natural killer cells|Non classical monocytes|Non-switched memory B cells|Non-Vd2 gd T cells|Plasmablasts|Plasmacytoid dendritic cells|Progenitor cells|Switched memory B cells|T regulatory cells|Terminal effector CD8 T cells|Th1 cells|Th1/Th17 cells|Th17 cells|Th2 cells|Vd2 gd T cells,#A6CEE3|#6CA9CF|#3385BB|#4693A8|#84BF96|#A3D77F|#6DBD57|#37A22F|#7F9D55|#D49B84|#F57C7C|#EB4647|#E42622|#F06C45|#FBB268|#FDA848|#FE8D19|#F48829|#DE9E83|#C6ADD3|#9D7BBA|#754AA0|#977899|#D6CA99|#F3E587|#D29F57|#B15928,7,TRUE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14,#A6CEE3|#3B8ABE|#72B29C|#84C868|#4F9F3B|#EC9A91|#E93E3F|#F06C45|#FDAC4F|#FB820F|#D1AAB7|#8C66AF|#A99099|#EEDB80|#B15928,4,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14,#A6CEE3|#3B8ABE|#72B29C|#84C868|#4F9F3B|#EC9A91|#E93E3F|#F06C45|#FDAC4F|#FB820F|#D1AAB7|#8C66AF|#A99099|#EEDB80|#B15928,4,TRUE,FALSE


In [48]:
# Controls
load(file=file.path('objects', '06.rda'))
obj <- subset(obj, subset = Stim == "healthy control")
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)

save(obj, file=file.path('objects', 'hc.rda'))

seu =  obj
scConf4 = createConfig(seu)
makeShinyFiles(seu, scConf4,
             gene.mapping = TRUE,
             shiny.prefix = "sc4",
             shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/",
             gex.assay = "SCT",
             default.multigene = present_genes) 

PC_ 1 
Positive:  IGHA1, IGKC, JCHAIN, IGHA2, IGLC2, IGLC3, IGHG1, IGLL5, IGHG3, MZB1 
	   CCL5, IGKV3-20, RGS1, CD69, HLA-DRA, IGKV4-1, CYBA, DERL3, KLRB1, IGHM 
	   CD52, CXCR4, IGLV2-14, CD3D, SPINK2, IGHG4, IGKV1-5, SRGN, IGLV6-57, TRBC1 
Negative:  ADAMDEC1, DCN, LUM, COL3A1, COL1A2, CCL2, C1S, CALD1, A2M, EGR1 
	   MFAP4, FN1, CXCL14, FOS, CFH, MEG3, C1R, FBLN1, TCF21, IFITM3 
	   CCL8, TMEM176B, GSN, EDIL3, ABCA8, NNMT, COL1A1, PTN, SELENOP, STMN2 
PC_ 2 
Positive:  KRT8, LGALS4, PHGR1, ELF3, FABP1, KRT18, PLCG2, FXYD3, PIGR, S100A6 
	   CLDN4, IFI27, KRT20, C15orf48, KRT19, SMIM22, MT1G, CLDN7, CA2, LGALS3 
	   TFF3, CKB, EPCAM, CA1, SLC26A3, MT1E, GUCA2A, TSPAN1, C19orf33, TSPAN8 
Negative:  IGHA1, JCHAIN, IGKC, IGHA2, IGLC2, ADAMDEC1, DCN, LUM, IGLC3, C1S 
	   COL3A1, COL1A2, CCL2, CALD1, MFAP4, A2M, FN1, CFH, TCF21, CCL8 
	   CXCL14, C1R, FBLN1, ABCA8, MEG3, EDIL3, PTN, IGLL5, COL1A1, NNMT 
PC_ 3 
Positive:  CCL5, TMSB4X, KLRB1, CXCR4, ANXA1, CD52, HLA-DRA, IL32, NKG7, GZMA 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 1745
Number of edges: 47254

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9214
Number of communities: 14
Elapsed time: 0 seconds


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,healthy control,black,1,FALSE,FALSE
Gender,Gender,Female|Male,#A6CEE3|#B15928,1,TRUE,FALSE
Orig.Annotations,Orig.Annotations,BEST4 OTOP2|Colonocyte 1|Colonocyte 2|Cycling cells|Cycling myeloid|Cycling TA|DN EOMES|DN TNF|Endothelium|Enteroendocrine|Epithelium Ribhi|Fibroblasts Ribhi|FRCs|gd IEL|Glia|Goblet|IER fibroblasts|Inflammatory colonocyte|Inflammatory fibroblasts|Inflammatory monocytes|Laminin colonocytes|Lymphatic endothelium|M2.2|MAIT|Mast 1|Mast 2|Mast Ribhi|Mature goblet|MT fibroblasts|MT T cells|Neutrophil 1|Neutrophil 2|Neutrophil 3|Paneth-like|PC immediate early response|PC IgG 2|PC IGLL5|Pericytes|Plasmablast IgG|Plasmablast IgG Lambda|PLCG2 colonocytes|S1|S1 Ribhi|S1.2|S2a|S2b|S3|Secretory progenitor,#A6CEE3|#86B9D8|#66A5CD|#4791C2|#277DB7|#3889AC|#5AA1A3|#7CB999|#9FD18F|#A4D880|#86C969|#69BA53|#4BAC3E|#3B9F30|#6A9E4A|#999C63|#C79B7D|#F69A96|#F57E7E|#F06061|#EA4243|#E52426|#E63228|#EC593B|#F37F4F|#F9A662|#FDB965|#FDAA4B|#FE9B31|#FE8C17|#FD8004|#F18C36|#E59768|#D8A39A|#CCAFCC|#B79BCA|#A180BC|#8A64AE|#7449A0|#7D5599|#9F8399|#C2B099|#E5DD99|#FAF491|#E7CD77|#D5A65C|#C37F42|#B15928,12,TRUE,FALSE
blueprint.main,blueprint.main,Adipocytes|B-cells|CD4+ T-cells|CD8+ T-cells|DC|Endothelial cells|Epithelial cells|Erythrocytes|Fibroblasts|HSC|Macrophages|Monocytes|NK cells,#A6CEE3|#2A7FB7|#99CD91|#52AF43|#B89B74|#ED4F50|#F06C45|#FDA440|#ED8F47|#B294C7|#825D99|#F8F18F|#B15928,4,TRUE,FALSE
monaco.fine,monaco.fine,Central memory CD8 T cells|Classical monocytes|Effector memory CD8 T cells|Exhausted B cells|Follicular helper T cells|Intermediate monocytes|Low-density basophils|MAIT cells|Myeloid dendritic cells|Naive B cells|Naive CD4 T cells|Natural killer cells|Non classical monocytes|Non-switched memory B cells|Non-Vd2 gd T cells|Plasmablasts|Plasmacytoid dendritic cells|Progenitor cells|Switched memory B cells|T regulatory cells|Terminal effector CD8 T cells|Th1 cells|Th1/Th17 cells|Th17 cells|Th2 cells|Vd2 gd T cells,#A6CEE3|#6AA8CE|#2F82B9|#4E98A6|#8EC694|#98D277|#60B64D|#439F34|#9B9C64|#F29A94|#F16667|#E62E30|#EA4833|#F59057|#FDB45D|#FE982C|#FC8108|#E59766|#CEADC4|#A787C0|#7D54A5|#8D6B99|#CFC099|#F5EB8B|#D3A259|#B15928,7,TRUE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13,#A6CEE3|#3385BB|#84BF96|#6DBD57|#7F9D55|#F57C7C|#E42622|#FBB268|#FE8D19|#DE9E83|#9D7BBA|#977899|#F3E587|#B15928,4,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13,#A6CEE3|#3385BB|#84BF96|#6DBD57|#7F9D55|#F57C7C|#E42622|#FBB268|#FE8D19|#DE9E83|#9D7BBA|#977899|#F3E587|#B15928,4,TRUE,FALSE


In [49]:
makeShinyCodesMulti(
  shiny.title = "UC and Crohn's IBD from colonic mucosa scRNAseq", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4"),
  shiny.headers = c("All", "UC", "CD", "Healthy"), 
  shiny.dir = "colon_UC_crohns_scRNA-seq_GSM6614348/") 

In [51]:
system("R -e \"shiny::runApp('colon_UC_crohns_scRNA-seq_GSM6614348')\"")